# Agentic Design Patterns - Routing

> *(Input → LLM Router → [Task-specific LLMs] → Output)*

> Using Ollama llama3.2 & phi4-mini models via LangChain


## Customer Sentiment Router 

**Goal:** Auto-route an inbound customer message to the best reply generator based on sentiment + intent.

**High-level Flow:** 


1. **Input (I):** raw message + minimal context (customer id/tier, last order, risk flags).
2. **LLM Router:** classifies (sentiment, intent, urgency) and outputs a route.
3. **Task-specific LLMs/tools:**
    - Apology/Recovery Bot (refund/credit, service failure)
    - Recommendation Bot (positive/neutral sentiment, purchase intent)
    - Escalation Triage Bot (angry, legal, churn-risk, safety)
4. **Output (O):** reply text + action payload (e.g., refund amount, ticket priority).

**Routing Targets & Criteria**
- Apology → sentiment ∈ [negative, frustrated}; intent ∈ [defect, late delivery, billing error}; urgency ≥ medium
- Recommendation → sentiment ∈ {positive, neutral}; intent ∈ {browse, upgrade, add-on}; no open incidents; LTV high.
- Escalation → sentiment ∈ {very negative}; intent ∈ {cancellation, chargeback, legal, safety}; toxic language or VIP


# Set up

## Imports

In [124]:
from dotenv import load_dotenv
import os, json
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain import LLMChain, PromptTemplate
from IPython.display import Markdown, display

## Load environment variables

In [ ]:
load_dotenv(override=True)

Check API Keys are loaded

In [ ]:
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')
OLLAMA_BASE_URL = os.getenv('OLLAMA_BASE_URL')
OLLAMA_MODEL_LLAMA = os.getenv('OLLAMA_MODEL_LLAMA')
OLLAMA_MODEL_PHI = os.getenv('OLLAMA_MODEL_PHI')

# Check Ollama
if OLLAMA_API_KEY and OLLAMA_BASE_URL and OLLAMA_MODEL_LLAMA and OLLAMA_MODEL_PHI:
    print(f"Ollama is set:")
    print(f"\t- OLLAMA_BASE_URL = {OLLAMA_BASE_URL}")
    print(f"\t- OLLAMA_MODEL_LLAMA = {OLLAMA_MODEL_LLAMA}")
    print(f"\t- OLLAMA_MODEL_PHI = {OLLAMA_MODEL_PHI}")
else:
    print("Ollama parameter(s) not set.")

## Ollama LLMs

In [ ]:
# Uncomment to pull models locally if needed
# !ollama pull llama3.2
# !ollama pull phi4-mini

**API call - Request and Response format**

```python

# LangChain ChatOpenAI with Ollama LLMs
model = ChatOpenAI(
    model=OLLAMA_MODEL_LLAMA, 
    base_url=OLLAMA_BASE_URL, 
    api_key=OLLAMA_API_KEY
)

# Messages can be of 2 formats:
# 1. Simple list of dicts
messages = [
    {"role": "system", "content": "text here"},
    {"role": "human", "content": "text here"},
]
# 2. LangChain Message objects
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage("You are a helpful assistant that translates English to French."),
    HumanMessage("Translate: I love programming."),
    AIMessage("J'adore la programmation."),
    HumanMessage("Translate: I love building applications.")
]

# Invoke the model
response = model.invoke(
    messages=messages
    )

display(Markdown(response)) // response is AIMessage content
```

---
# Implementation

Create LLM objects for router and task-specific bots.

In [29]:
# Currently using Ollama for all roles. Update models as needed.
params = {
    "temperature": 0.7,
    "max_tokens": 1000
}

LLMs = {"ROUTER": ChatOpenAI(model=OLLAMA_MODEL_LLAMA, base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY, temperature=0.0, max_tokens=1000),
        "BOTS": {
            "APOLOGY": ChatOpenAI(model=OLLAMA_MODEL_LLAMA, base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY, **params),
            "RECOMMENDATION": ChatOpenAI(model=OLLAMA_MODEL_LLAMA, base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY, **params),
            "ESCALATION": ChatOpenAI(model=OLLAMA_MODEL_LLAMA, base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY, **params)
            },
        "DEFAULT": ChatOpenAI(model=OLLAMA_MODEL_PHI, base_url=OLLAMA_BASE_URL, api_key=OLLAMA_API_KEY, **params)
        }


## Task-specific Bots (specialized LLMs)

In [184]:
class LLMBot:
    def __init__(self): 
        self.model = None
        self.response = None
        self.out_json = None
 
    def build_messages(self, router_response: dict) -> list:
        pass
    
    def get_raw_response(self, messages: list) -> AIMessage:
        pass
 
    def handle(self, router_response: dict, context: dict)-> dict:
        pass

### Apology/Recovery Bot

In [185]:
template_apo = PromptTemplate(
    input_variables=["router_intent", "out_json"],
    template="""
    Issue summary: "{router_intent} - <short reason>
    Policy: full refund if defect, partial credit if delay < 48h.
    Write 60-90 words, sincere tone. Include next step + exact timeline hours.
    Output JSON: {out_json}
    """
)

out_json_apo = {
    "reply": "...", 
    "action": {
        "type": "refund|credit|reship",
        "amount": "...", 
        "sla_hours": 24
    }
}

In [186]:
class ApologyBot(LLMBot):
    def __init__(self): 
        self.model = LLMs.get("BOTS").get("APOLOGY")
        self.response = None
        self.out_json = out_json_apo

    def build_messages(self, router_response: dict) -> list:
        system_msg = "You write concise, sincere recovery messages and propose remedies. Return JSON only. No markdown."
        user_msg = template_apo.format(
            router_intent=router_response.get("intent", "unknown"),
            out_json=json.dumps(self.out_json)
        )
        
        return [
            SystemMessage(content=system_msg),
            HumanMessage(content=user_msg)
        ]

    def get_raw_response(self, messages: list) -> AIMessage:
        return self.model.invoke(messages=messages)

    def handle(self, router_response: dict)-> dict:
        messages = self.build_messages(router_response)
        response = self.get_raw_response(messages).content
        self.response = json.loads(response) if isinstance(response, str) else response
        return self.response

### Recommendation Bot

In [187]:
out_json_rec = {
            "reply": "...", 
            "offers": [
                {"sku": "...", "reason": "..."},
            ]
        }

template_rec = PromptTemplate(
    input_variables=["out_json"],
    template="""
        Customer profile: {{"tier":"Gold", "past_purchases":["Starter Plan"], "ltv":1200}}
        Goal: suggest 1 upgrade and 1 add-on; cite 1 benefit each; 1 CTA max.
        Output JSON: {out_json}
        """
)

In [212]:
class RecommendationBot(LLMBot):
    def __init__(self): 
        self.model = LLMs.get("BOTS").get("RECOMMENDATION")
        self.response = None
        self.out_json = out_json_rec

    def build_messages(self, router_response=None) -> list:
        system_msg = "You recommend products based on customer issues. Return JSON only. No markdown."
        user_msg = template_rec.format(
            out_json=json.dumps(self.out_json)
        )
        
        return [
            SystemMessage(content=system_msg),
            HumanMessage(content=user_msg)
        ]
    
    def get_raw_response(self, messages: list) -> AIMessage:
        return self.model.invoke(input=messages)
    
    def handle(self, router_response=None)-> dict:
        messages = self.build_messages()
        response = self.get_raw_response(messages).content
        self.response = json.loads(response) if isinstance(response, str) else response
        return self.response

### Escalation Triage Bot

In [189]:
out_json_esc = {
        "reply": "...", 
        "ticket": {
            "priority": "P1|P2",
            "category": "billing|logistics|legal",
            "required_fields": ["order_id", "photos"]
        }
    }

template_esc = PromptTemplate(
    input_variables=["router_intent", "out_json"],
    template="""
        Intent: "{router_intent}"
        Do: acknowledge; take ownership; ask exactly 2 clarifying questions; promise human follow-up with a timeline.
        Ticket: P1 if very_negative or intent in ["cancellation", "safety"], else P2; category from intent; required_fields must include "order_id".
        Output JSON: {out_json}
        """
)

In [202]:
class EscalationBot(LLMBot):
    def __init__(self): 
        self.model = LLMs.get("BOTS").get("ESCALATION")
        self.response = None
        self.out_json = out_json_esc

    def build_messages(self, router_response: dict) -> list:
        system_msg = "You draft escalation messages and create support tickets. Return JSON only. No markdown."
        user_msg = template_esc.format(
            router_intent=router_response.get("intent", "unknown"),
            out_json=json.dumps(self.out_json)
        )
        
        return [
            SystemMessage(content=system_msg),
            HumanMessage(content=user_msg)
        ]
    
    def get_raw_response(self, messages: list) -> AIMessage:
        return self.model.invoke(input=messages)
    
    def handle(self, router_response: dict)-> dict:
        messages = self.build_messages(router_response)
        response = self.get_raw_response(messages).content
        self.response = json.loads(response) if isinstance(response, str) else response
        return self.response

## Router LLM

In [191]:
template_router = PromptTemplate(
    input_variables=["customer_msg", "out_json"],
    template = """
    You classify the customer message. Return JSON only (no markdown).
    Message: {customer_msg}
    Context: {{"customer_tier":"Gold", "open_ticket":false, "ltv":1200}}
    Labels wanted: 
        - sentiment in [very_negative, negative, neutral, positive], 
        - intent in [billing_issue, delivery_issue, product_defect, cancellation, purchase_intent, general_question, safety],
        - urgency in [low, med, high].

    Routing rules (evaluate in order; first match wins):
        1) Escalation (hard triggers)
            - text contains any of ["cancel", "chargeback", "lawyer", "attorney", "legal", "sue", "report you", "fraud", "scam", "bbb", "ripoff", "threat"]
            - OR intent in [cancellation, safety]
            - OR sentiment == very_negative
        2) Apology (service recovery)
            - sentiment == negative
            - AND intent in [billing_issue, delivery_issue, product_defect]
        3) Recommendation (default)
            - sentiment in [neutral, positive]
            - OR intent in [purchase_intent, general_question]
    
    Output JSON: {out_json}
    """
)

out_json_router = {
    "sentiment": "...",
    "intent": "...",
    "urgency": "...",
    "route": "apology|recommendation|escalation"
}

In [213]:
class Router:
    def __init__(self):
        self.model = LLMs.get("ROUTER")
        self.out_json = out_json_router
        self.response = None
        self.route = None

        # Initialize bots
        self.apology_bot = ApologyBot()
        self.recommendation_bot = RecommendationBot()
        self.escalation_bot = EscalationBot()

    def build_messages(self, customer_msg: str) -> list:
        system_msg = "You classify customer messages. Return Output JSON only. No markdown."
        user_msg = template_router.format(
            customer_msg=customer_msg, 
            out_json=json.dumps(self.out_json)
            )

        return [
            (SystemMessage(content=system_msg)),
            (HumanMessage(content=user_msg))
        ]

    def get_raw_response(self, messages) -> AIMessage:
        return self.model.invoke(input=messages)

    def classify(self) -> str:
        return self.response.get("route", "escalation")

    def handle(self, customer_msg, context={}):
        # Get router response & classify 
        messages = self.build_messages(customer_msg)
        response = (self.get_raw_response(messages)).content
        self.response = json.loads(response) if isinstance(response, str) else response
        self.route = self.classify()

        # # Route to appropriate bot
        if self.route == "apology":
            return self.apology_bot.handle(self.response)
        elif self.route == "recommendation":
            return self.recommendation_bot.handle(self.response)
        else:
            return self.escalation_bot.handle(self.response)
        


## Generic Bot

In [193]:
class ReplyCustomer:
    def __init__(self):
        self.model = LLMs.get("DEFAULT")
        self.messages = None

    def build_messages(self, bot_intent, bot_response: dict):
        system_msg = "You are a customer support agent. Write a clear, friendly reply based on given context. Return reply only."

        # Keep only the pieces the writer needs
        minimal_context = {
            "bot_type": bot_intent,
            "reply_hint": bot_response.get("reply", ""),
            "action": bot_response.get("action", {}),
            "ticket": bot_response.get("ticket", {}),
            "offers": bot_response.get("offers", []),
        }

        # One compact instruction that works for all cases
        user_msg = f"""
        Craft the final customer-facing message using the BOT_SUMMARY as context.
        BOT_SUMMARY: {minimal_context}

        Rules:
            - 50–100 words, friendly and professional.
            - If bot_type is: 
                - 'apology': apologize briefly, state remedy/next step, give exact timeline (hours), and politely ask for missing required_fields if any.
                - 'escalation': acknowledge, de-escalate, ask exactly 2 concise questions (if needed), state that a specialist will follow up and provide a realistic timeline (hours).
                - 'recommendation': thank the customer and present exactly 1 upgrade and 1 add-on with a short benefit and one simple CTA.
            - If no name is provided, sign off as 'Customer Service Team'.

        Reply: <add your reply here>
        Return reply only.
        """

        self.messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ]

        return self.messages

    def get_response(self, messages) -> str:        
        raw_response = self.model.invoke(input=messages)
        return raw_response.content.strip()

    def handle(self, bot_intent, bot_response: dict) -> str:
        self.build_messages(bot_intent, bot_response)
        self.response = self.get_response(self.messages)
        return self.response

# Testing

In [214]:
router = Router()
replier = ReplyCustomer()

In [205]:
# Test messages for each route
customer_msg_1 = "I just received my order, but it arrived late and one item is missing."
customer_msg_2 = "So far, I'm happy with my Starter Plan. I would like to know what upgrades or add-ons you would recommend."
customer_msg_3 = "The product is defective. I will sue you if you don't refund me immediately!"

## Test Case: ApologyBot

In [208]:
router_resp_1.get("route")

'apology|recommendation|escalation'

In [209]:
bot_resp_1 = router.handle(customer_msg_1)
router_resp_1 = router.response
reply_1 = replier.handle(router_resp_1.get("route"), bot_resp_1)

In [210]:
display(Markdown(f"**Customer Message:**\n```\n{customer_msg_1}\n```"))
display(Markdown(f"**Router Response:**\n```\n{json.dumps(router_resp_1, indent=2)}\n```"))
display(Markdown(f"**Final Reply:**\n```\n{reply_1}\n```"))

**Customer Message:**
```
I just received my order, but it arrived late and one item is missing.
```

**Router Response:**
```
{
  "sentiment": "negative",
  "intent": "delivery_issue",
  "urgency": "low",
  "route": "apology|recommendation|escalation"
}
```

**Final Reply:**
```
Dear Customer,

We are truly sorry to hear that there was an issue with the delivery of Order ID #12345. We understand how frustrating this can be and apologize for any inconvenience caused.

Could you please confirm if:
1. The problem occurred during pickup or after it had been handed over?
2. Were photos provided as required?

A specialist will follow up on your case within 24 hours to resolve the issue promptly, ensuring that we meet our high standards of service with every order placed through us again in confidence.

Thank you for choosing [Your Company Name]. We appreciate your understanding and cooperation during this time.
Customer Service Team
```

## Test Case: RecommendationBot

In [215]:
bot_resp_2 = router.handle(customer_msg_2, context={})
router_resp_2 = router.response
reply_2 = replier.handle(router_resp_2.get("route"), bot_resp_2)

In [216]:
display(Markdown(f"**Customer Message:**\n```\n{customer_msg_2}\n```"))
display(Markdown(f"**Router Response:**\n```\n{json.dumps(router_resp_2, indent=2)}\n```"))
display(Markdown(f"**Final Reply:**\n```\n{reply_2}\n```"))

**Customer Message:**
```
So far, I'm happy with my Starter Plan. I would like to know what upgrades or add-ons you would recommend.
```

**Router Response:**
```
{
  "sentiment": "positive",
  "intent": "general_question",
  "urgency": "low",
  "route": "recommendation"
}
```

**Final Reply:**
```
Thank you for reaching out to us! Based on the details in our system about how often you're using various tools like Microsoft Office 365 and Adobe Creative Cloud subscriptions (MacBook Pro), we recommend upgrading to our Professional Plan. This will enhance productivity with more features, giving your work increased efficiency and accuracy.

Additionally, consider adding a Premium Support Add-on for personalized assistance from experts around the clock - perfect when you need help without interrupting others or waiting in queues!

If you'd like further information on these options: (1) What's included? (2) How do they compare to our Basic Plan?

Upgrade Now.
```

## Test Case: EscalationBot

In [217]:
bot_resp_3 = router.handle(customer_msg_3, context={})
router_resp_3 = router.response
reply_3 = replier.handle(router_resp_3.get("route"), bot_resp_3)

In [218]:
display(Markdown(f"**Customer Message:**\n```\n{customer_msg_3}\n```"))
display(Markdown(f"**Router Response:**\n```\n{json.dumps(router_resp_3, indent=2)}\n```"))
display(Markdown(f"**Final Reply:**\n```\n{reply_3}\n```"))

**Customer Message:**
```
The product is defective. I will sue you if you don't refund me immediately!
```

**Router Response:**
```
{
  "sentiment": "very_negative",
  "intent": "product_defect",
  "urgency": "high",
  "route": "escalation"
}
```

**Final Reply:**
```
Thank you for reporting this issue. I am taking ownership of the ticket to resolve it promptly.

Could you please provide us with:
1. The order ID associated with these issues?
2. If available, any photos that illustrate what you've encountered?

A specialist will follow up on your concerns within 24 hours and work diligently towards a resolution for both Priority P1 and P2 tickets in the billing category.



Thank you so much! As an upgrade to enhance our service experience further (and possibly save some time), we highly recommend [Upgrade Name] with its key feature of faster processing. Additionally, if you'd like more convenience at home or on-the-go, consider adding [Add-On Name], which provides amazing benefits for your daily needs.

If there's anything else I can assist you with today apart from this issue!
```